In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np

In [ ]:
def resize_with_padding(image, desired_size):
    old_size = image.shape[:2]
    ratio = float(desired_size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])

    # 리사이즈
    image = cv2.resize(image, (new_size[1], new_size[0]))

    # 패딩 추가
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    color = [0, 0, 0]
    new_image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

    return new_image

In [ ]:
# 원본 비디오 데이터 경로 & 저장할 이미지 폴더 경로
video_root = "/content/drive/MyDrive/DMS/data/YawDD dataset/clip_with_adjusted_label/"  # 비디오 데이터 폴더
output_root = "/content/drive/MyDrive/DMS/data/YawDD dataset/YawDD_adjusted_frames_resized/"  # 프레임이 저장될 폴더

# 비디오 파일을 찾고 프레임을 추출하는 함수
def extract_frames_from_video(video_path, output_folder):
    os.makedirs(output_folder, exist_ok=True)  # 폴더가 없으면 생성
    # 사용할 프레임 설정
    frame_interval = 5  # 5프레임마다 1개씩 저장
    resize_shape = 224  # 이미지 크기 조정

    cap = cv2.VideoCapture(video_path)  # 비디오 파일 읽기
    if not cap.isOpened():
        print(f"⚠️ Could not open video: {video_path}")
        return

    frame_count = 0
    saved_frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # 비디오 끝에 도달하면 종료

        if frame_count % frame_interval == 0:  # 지정된 간격마다 프레임 저장
            frame = resize_with_padding(frame, resize_shape)  # 크기 조정
            frame_filename = os.path.join(output_folder, f"frame{saved_frame_count:06d}.jpg")
            cv2.imwrite(frame_filename, frame)  # 이미지 저장
            saved_frame_count += 1

        frame_count += 1

    cap.release()
    print(f"✅ Extracted {saved_frame_count} frames from {video_path}")

# Clipped 폴더 내 모든 파일 처리
def process_clipped_videos(clipped_root, output_root):
    subcategories = ["Normal", "Talking", "Yawning"]
    categories = os.listdir(clipped_root)

    for category in categories:
        category_path = os.path.join(clipped_root, category)
        for subfolder in subcategories:
            subfolder_path = os.path.join(category_path, subfolder)

            if not os.path.exists(subfolder_path):
                print(f"⚠️ Skipping missing folder: {subfolder_path}")
                continue  # 폴더가 없으면 건너뛰기

            for filename in os.listdir(subfolder_path):
                if filename.endswith(".avi"):  # 비디오 파일만 처리
                    video_path = os.path.join(subfolder_path, filename)
                    output_folder = os.path.join(output_root, category, subfolder, os.path.splitext(filename)[0])

                    extract_frames_from_video(video_path, output_folder)  # 프레임 추출

In [ ]:
process_clipped_videos(video_root, output_root)

✅ Extracted 48 frames from /content/drive/MyDrive/DMS/data/YawDD dataset/clip_with_adjusted_label/dash_female_split_output_adjusted/Yawning/1-FemaleNoGlasses-clip-0.avi
✅ Extracted 48 frames from /content/drive/MyDrive/DMS/data/YawDD dataset/clip_with_adjusted_label/dash_female_split_output_adjusted/Yawning/1-FemaleNoGlasses-clip-1.avi
✅ Extracted 36 frames from /content/drive/MyDrive/DMS/data/YawDD dataset/clip_with_adjusted_label/dash_female_split_output_adjusted/Yawning/2-FemaleNoGlasses-clip-1.avi
✅ Extracted 18 frames from /content/drive/MyDrive/DMS/data/YawDD dataset/clip_with_adjusted_label/dash_female_split_output_adjusted/Yawning/2-FemaleNoGlasses-clip-0.avi
✅ Extracted 30 frames from /content/drive/MyDrive/DMS/data/YawDD dataset/clip_with_adjusted_label/dash_female_split_output_adjusted/Yawning/3-FemaleGlasses-clip-0.avi
✅ Extracted 42 frames from /content/drive/MyDrive/DMS/data/YawDD dataset/clip_with_adjusted_label/dash_female_split_output_adjusted/Yawning/3-FemaleGlasses-c

In [ ]:
# dmd_labeled 내 모든 파일 처리
def process_clipped_videos(clipped_root, output_root):
    labels = ["Normal", "PhoneCall", "Text"]

    for label in labels:
        label_path = os.path.join(clipped_root, label)

        if not os.path.exists(label_path):
            print(f"⚠️ Skipping missing folder: {label_path}")
            continue  # 폴더가 없으면 건너뛰기

        for filename in os.listdir(label_path):
            if filename.endswith(".avi"):  # 비디오 파일만 처리
                video_path = os.path.join(label_path, filename)
                output_folder = os.path.join(output_root, label, filename)

                extract_frames_from_video(video_path, output_folder)  # 프레임 추출

dmd_video = "/content/drive/MyDrive/DMS/data/DMD/dmd_labeled/"  # 비디오 데이터 폴더
dmd_output = "/content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/"  # 프레임이 저장될 폴더

process_clipped_videos(dmd_video, dmd_output)

✅ Extracted 19 frames from /content/drive/MyDrive/DMS/data/DMD/dmd_labeled/Normal/gZ-34(File28)_s4_face_2019-04-02-10;01;53_34_27.avi
✅ Extracted 23 frames from /content/drive/MyDrive/DMS/data/DMD/dmd_labeled/Normal/gZ-34(File28)_s4_face_2019-04-03-09;47;42_34_24.avi
✅ Extracted 10 frames from /content/drive/MyDrive/DMS/data/DMD/dmd_labeled/Normal/gZ-34(File28)_s4_face_2019-04-03-09;47;42_34_13.avi
✅ Extracted 26 frames from /content/drive/MyDrive/DMS/data/DMD/dmd_labeled/Normal/gZ-34(File28)_s4_face_2019-04-02-10;01;53_34_21.avi
✅ Extracted 37 frames from /content/drive/MyDrive/DMS/data/DMD/dmd_labeled/Normal/gZ-34(File28)_s4_face_2019-04-02-10;01;53_34_12.avi
✅ Extracted 9 frames from /content/drive/MyDrive/DMS/data/DMD/dmd_labeled/Normal/gZ-34(File28)_s4_face_2019-04-03-09;47;42_34_29.avi
✅ Extracted 25 frames from /content/drive/MyDrive/DMS/data/DMD/dmd_labeled/Normal/gZ-34(File28)_s4_face_2019-04-02-10;01;53_34_26.avi
✅ Extracted 6 frames from /content/drive/MyDrive/DMS/data/DMD/d

In [ ]:
# dmd_labeled 내 모든 파일 처리
def process_clipped_videos(dmd_video, output_root):
    label = 'yawn'

    for dirpath, dirnames, filenames in os.walk(dmd_video):
        for file in filenames:
            if file.lower().endswith('.avi'):
                full_path = os.path.join(dirpath, file)
                output_folder = os.path.join(output_root, label, file)

                extract_frames_from_video(full_path, output_folder)  # 프레임 추출

dmd_video = "/content/drive/MyDrive/DMS/data/DMD_yawn/"  # 비디오 데이터 폴더
dmd_output = "/content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/"  # 프레임이 저장될 폴더

process_clipped_videos(dmd_video, dmd_output)

✅ Extracted 33 frames from /content/drive/MyDrive/DMS/data/DMD_yawn/gA-1(File1)/s5/yawning/Yawning with hand/body_2019-03-14-14;26;17_1_0.avi
✅ Extracted 38 frames from /content/drive/MyDrive/DMS/data/DMD_yawn/gA-1(File1)/s5/yawning/Yawning with hand/body_2019-03-14-14;26;17_1_1.avi
✅ Extracted 33 frames from /content/drive/MyDrive/DMS/data/DMD_yawn/gA-1(File1)/s5/yawning/Yawning with hand/face_2019-03-14-14;26;17_1_0.avi
✅ Extracted 38 frames from /content/drive/MyDrive/DMS/data/DMD_yawn/gA-1(File1)/s5/yawning/Yawning with hand/face_2019-03-14-14;26;17_1_1.avi
✅ Extracted 40 frames from /content/drive/MyDrive/DMS/data/DMD_yawn/gA-1(File1)/s5/yawning/Yawning without hand/body_2019-03-14-14;26;17_1_0.avi
✅ Extracted 39 frames from /content/drive/MyDrive/DMS/data/DMD_yawn/gA-1(File1)/s5/yawning/Yawning without hand/body_2019-03-14-14;26;17_1_2.avi
✅ Extracted 40 frames from /content/drive/MyDrive/DMS/data/DMD_yawn/gA-1(File1)/s5/yawning/Yawning without hand/face_2019-03-14-14;26;17_1_0.a

KeyboardInterrupt: 

In [ ]:
label = 'yawn'

for dirpath, dirnames, filenames in os.walk(dmd_video):
    for file in filenames:
        if file.lower().endswith('.avi'):
            full_path = os.path.join(dirpath, file)
            output_folder = os.path.join(dmd_output, label, file)

            extract_frames_from_video(full_path, output_folder)  # 프레임 추출
            print(output_folder)

✅ Extracted 33 frames from /content/drive/MyDrive/DMS/data/DMD_yawn/gA-1(File1)/s5/yawning/Yawning with hand/body_2019-03-14-14;26;17_1_0.avi
/content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/body_2019-03-14-14;26;17_1_0.avi
✅ Extracted 38 frames from /content/drive/MyDrive/DMS/data/DMD_yawn/gA-1(File1)/s5/yawning/Yawning with hand/body_2019-03-14-14;26;17_1_1.avi
/content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/body_2019-03-14-14;26;17_1_1.avi
✅ Extracted 33 frames from /content/drive/MyDrive/DMS/data/DMD_yawn/gA-1(File1)/s5/yawning/Yawning with hand/face_2019-03-14-14;26;17_1_0.avi
/content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/face_2019-03-14-14;26;17_1_0.avi
✅ Extracted 38 frames from /content/drive/MyDrive/DMS/data/DMD_yawn/gA-1(File1)/s5/yawning/Yawning with hand/face_2019-03-14-14;26;17_1_1.avi
/content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/face_2019-03-14-14;26;17_1_1.avi
✅ Extracted 40 frames from /content/drive/MyDrive/DMS/data/D

In [1]:
import os
# 예시 경로 (내 드라이브에 추가된 경우)
root_path = "/content/drive/MyDrive/DMS/data/processed/dmd_frames_resized/Yawn"

# 폴더 구조 탐색

for dirpath, dirnames, filenames in os.walk(root_path):
    print(f"📁 {dirpath}")
    for filename in filenames:
        print(f"   └── {filename}")

In [ ]:
import shutil
# body 삭제
dmd_yawn = "/content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/"

for folder_name in os.listdir(dmd_yawn):
    folder_path = os.path.join(dmd_yawn, folder_name)
    # 폴더인지 확인하고 이름이 'body'로 시작하는지 확인
    if os.path.isdir(folder_path) and folder_name.startswith("body"):
        shutil.rmtree(folder_path) # older_path가 가리키는 디렉토리 자체와 그 안에 있는 모든 파일삭제
        print(f"Deleted: {folder_path}")

Deleted: /content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/body_2019-03-14-14;26;17_1_0.avi
Deleted: /content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/body_2019-03-14-14;26;17_1_1.avi
Deleted: /content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/body_2019-03-14-14;26;17_1_2.avi
Deleted: /content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/body_2019-03-13-09;19;23_2_0.avi
Deleted: /content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/body_2019-03-13-09;19;23_2_2.avi
Deleted: /content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/body_2019-03-13-09;19;23_2_3.avi
Deleted: /content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/body_2019-03-13-09;19;23_2_1.avi
Deleted: /content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/body_2019-03-13-09;36;25_3_0.avi
Deleted: /content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/body_2019-03-13-09;36;25_3_1.avi
Deleted: /content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/yawn/body_2019-03-